# Leitura das Bibliotecas utilizadas no projeto

In [1]:
# Análise e tratamento de dados
import pandas as pd
import numpy as np
import json

# Requisição via API
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Banco de dados
import sqlite3

# Visualização dos dados
import matplotlib.pyplot as plt

# Regressão Linear
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Etapa 01 - Acessar dados sobre a Receita Municipal

## Requisitar dados do site do Tribunal de Contas do Estado de Pernambuco

In [2]:
# Obs: etapa inicial, preparatória e necessária para não ter dificuldades com requisições ao site do TCE-PE

session = requests.Session()
retry = Retry(connect=5, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [3]:
# Solicita dados do site do TCE-PE via API
exercicios = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
dicionario = {}
base_dados = []
base_dados_temp = []
for i, ano in enumerate(exercicios):
  url = 'https://sistemas.tce.pe.gov.br/DadosAbertos/ReceitasMunicipais!json?ANOREFERENCIA=%s&ID_UNIDADE_GESTORA=319' % (ano)
  resposta = session.get(url, stream=True)
  if i == 0:
    dicionario = json.loads(resposta.text)
    base_dados = pd.DataFrame.from_dict(data=dicionario['resposta']['conteudo'])
    print('Ano {} concluído.'.format(ano))
    print(f'Ano {ano}: {base_dados.shape[0]} linhas carregadas.')
  else:
    dicionario = json.loads(resposta.text)
    base_dados_temp = pd.DataFrame.from_dict(data=dicionario['resposta']['conteudo'])
    print('Ano {} concluído.'.format(ano))
    print('\n')
    print(f'Ano {ano}: {base_dados_temp.shape[0]} linhas carregadas.')
    base_dados = pd.concat([base_dados, base_dados_temp])
    print(f'Total: {base_dados.shape[0]} linhas carregadas.')
    

Ano 2012 concluído.
Ano 2012: 924 linhas carregadas.
Ano 2013 concluído.


Ano 2013: 972 linhas carregadas.
Total: 1896 linhas carregadas.
Ano 2014 concluído.


Ano 2014: 1011 linhas carregadas.
Total: 2907 linhas carregadas.
Ano 2015 concluído.


Ano 2015: 1248 linhas carregadas.
Total: 4155 linhas carregadas.
Ano 2016 concluído.


Ano 2016: 1376 linhas carregadas.
Total: 5531 linhas carregadas.
Ano 2017 concluído.


Ano 2017: 1305 linhas carregadas.
Total: 6836 linhas carregadas.
Ano 2018 concluído.


Ano 2018: 1347 linhas carregadas.
Total: 8183 linhas carregadas.
Ano 2019 concluído.


Ano 2019: 1428 linhas carregadas.
Total: 9611 linhas carregadas.
Ano 2020 concluído.


Ano 2020: 1892 linhas carregadas.
Total: 11503 linhas carregadas.
Ano 2021 concluído.


Ano 2021: 1902 linhas carregadas.
Total: 13405 linhas carregadas.
Ano 2022 concluído.


Ano 2022: 1001 linhas carregadas.
Total: 14406 linhas carregadas.


## Análise e Tratamento dos dados requisitados

In [4]:
base_dados.shape

(14406, 12)

In [5]:
base_dados.columns

Index(['ID_UNIDADE_GESTORA', 'NUMERORECEITAORCAMENTARIAUG', 'DESCRICAO',
       'CATEGORIARECEITA', 'ANOREFERENCIA', 'MESREFERENCIA', 'VALORLANCAMENTO',
       'ESFERA', 'ORIGEMRECEITA', 'DEDUCAORECEITA', 'ESPECIERECEITA',
       'DESCRICAOFONTE'],
      dtype='object')

In [6]:
base_dados = base_dados.reset_index()

In [7]:
base_dados = base_dados.drop(labels='index', axis=1)

In [8]:
base_dados['COMPETENCIA'] = pd.to_datetime({'year': base_dados['ANOREFERENCIA'], 'month': base_dados['MESREFERENCIA'], 'day': 1}, format="%m/%Y")

In [9]:
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14406 entries, 0 to 14405
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID_UNIDADE_GESTORA           14406 non-null  object        
 1   NUMERORECEITAORCAMENTARIAUG  14404 non-null  object        
 2   DESCRICAO                    14404 non-null  object        
 3   CATEGORIARECEITA             14406 non-null  object        
 4   ANOREFERENCIA                14406 non-null  object        
 5   MESREFERENCIA                14406 non-null  object        
 6   VALORLANCAMENTO              14406 non-null  object        
 7   ESFERA                       14406 non-null  object        
 8   ORIGEMRECEITA                14404 non-null  object        
 9   DEDUCAORECEITA               14406 non-null  object        
 10  ESPECIERECEITA               13005 non-null  object        
 11  DESCRICAOFONTE               12510 non-nu

In [10]:
base_dados.head()

,ID_UNIDADE_GESTORA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,CATEGORIARECEITA,ANOREFERENCIA,MESREFERENCIA,VALORLANCAMENTO,ESFERA,ORIGEMRECEITA,DEDUCAORECEITA,ESPECIERECEITA,DESCRICAOFONTE,COMPETENCIA
0,319,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,Dedução de Receitas,2012,1,-4810177.11,M,Alienação de Bens,9,NaN,NaN,2012-01-01
1,319,917220102,Dedução de Cota-Parte do IPVA - FUNDEB,Dedução de Receitas,2012,1,-145288.61,M,Alienação de Bens,9,NaN,NaN,2012-01-01
2,319,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,Dedução de Receitas,2012,1,-4222.80,M,Alienação de Bens,9,NaN,NaN,2012-01-01
3,319,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,Dedução de Receitas,2012,2,-8771.62,M,Alienação de Bens,9,NaN,NaN,2012-02-01
4,319,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,Dedução de Receitas,2012,3,-6409.60,M,Alienação de Bens,9,NaN,NaN,2012-03-01


In [11]:
# Lista de Receitas e Códigos
receitas_cod = base_dados[['NUMERORECEITAORCAMENTARIAUG', 'DESCRICAO']].drop_duplicates()
receitas_cod.to_csv('receitas_cod.csv', sep=';', index=False)

In [12]:
base_dados.loc[base_dados['NUMERORECEITAORCAMENTARIAUG'].isnull()]

,ID_UNIDADE_GESTORA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,CATEGORIARECEITA,ANOREFERENCIA,MESREFERENCIA,VALORLANCAMENTO,ESFERA,ORIGEMRECEITA,DEDUCAORECEITA,ESPECIERECEITA,DESCRICAOFONTE,COMPETENCIA
2925,319,NaN,NaN,Receitas Correntes,2015,11,653831.00,M,NaN,0,NaN,Outras Fontes,2015-11-01
3051,319,NaN,NaN,Receitas Correntes,2015,11,100000.00,M,NaN,0,NaN,Outras Fontes,2015-11-01


In [13]:
base_dados.drop(base_dados[base_dados['NUMERORECEITAORCAMENTARIAUG'].isnull()].index, inplace=True)

In [14]:
base_dados.to_csv('/content/base_dados')

In [15]:
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14404 entries, 0 to 14405
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID_UNIDADE_GESTORA           14404 non-null  object        
 1   NUMERORECEITAORCAMENTARIAUG  14404 non-null  object        
 2   DESCRICAO                    14404 non-null  object        
 3   CATEGORIARECEITA             14404 non-null  object        
 4   ANOREFERENCIA                14404 non-null  object        
 5   MESREFERENCIA                14404 non-null  object        
 6   VALORLANCAMENTO              14404 non-null  object        
 7   ESFERA                       14404 non-null  object        
 8   ORIGEMRECEITA                14404 non-null  object        
 9   DEDUCAORECEITA               14404 non-null  object        
 10  ESPECIERECEITA               13005 non-null  object        
 11  DESCRICAOFONTE               12508 non-nu

In [16]:
# Transformar numeros em valores flutuantes
base_dados['VALORLANCAMENTO'] = base_dados['VALORLANCAMENTO'].astype(float)
base_dados['NUMERORECEITAORCAMENTARIAUG'] = base_dados['NUMERORECEITAORCAMENTARIAUG'].astype(int)

In [17]:
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14404 entries, 0 to 14405
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID_UNIDADE_GESTORA           14404 non-null  object        
 1   NUMERORECEITAORCAMENTARIAUG  14404 non-null  int64         
 2   DESCRICAO                    14404 non-null  object        
 3   CATEGORIARECEITA             14404 non-null  object        
 4   ANOREFERENCIA                14404 non-null  object        
 5   MESREFERENCIA                14404 non-null  object        
 6   VALORLANCAMENTO              14404 non-null  float64       
 7   ESFERA                       14404 non-null  object        
 8   ORIGEMRECEITA                14404 non-null  object        
 9   DEDUCAORECEITA               14404 non-null  object        
 10  ESPECIERECEITA               13005 non-null  object        
 11  DESCRICAOFONTE               12508 non-nu

## Gravar dados requisitados em um Banco de Dados

In [18]:
base_dados_gravar = base_dados[['COMPETENCIA', 'ANOREFERENCIA', 'ORIGEMRECEITA', 'DEDUCAORECEITA', 'NUMERORECEITAORCAMENTARIAUG', 'DESCRICAO', 'VALORLANCAMENTO']]

In [19]:
base_dados_gravar.head()

,COMPETENCIA,ANOREFERENCIA,ORIGEMRECEITA,DEDUCAORECEITA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,VALORLANCAMENTO
0,2012-01-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-4810177.11
1,2012-01-01,2012,Alienação de Bens,9,917220102,Dedução de Cota-Parte do IPVA - FUNDEB,-145288.61
2,2012-01-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-4222.80
3,2012-02-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-8771.62
4,2012-03-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-6409.60


In [20]:
base_dados_gravar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14404 entries, 0 to 14405
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   COMPETENCIA                  14404 non-null  datetime64[ns]
 1   ANOREFERENCIA                14404 non-null  object        
 2   ORIGEMRECEITA                14404 non-null  object        
 3   DEDUCAORECEITA               14404 non-null  object        
 4   NUMERORECEITAORCAMENTARIAUG  14404 non-null  int64         
 5   DESCRICAO                    14404 non-null  object        
 6   VALORLANCAMENTO              14404 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 900.2+ KB


In [21]:
# Transformar data em texto para armazenamento no banco de dados - hipótese de explicação do erro.
base_dados_gravar['COMPETENCIA'] = base_dados_gravar['COMPETENCIA'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
# Criar conexão
con = sqlite3.connect('baseTransferencias.db')

# Criar cursor
cur = con.cursor()

In [23]:
#Inserir DE-PARA receitas no banco de dados - Fazer upload do arquivo.
receitas_tratadas = pd.read_csv('/content/receitas_tratadas.csv', sep=';', encoding='ISO-8859–1')

In [24]:
receitas_tratadas.head()

,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,CLASSIFICACAO_SEREC,DETALHAMENTO
0,917220101,Deducao de Cota-Parte do ICMS - FUNDEB,ICMS,DEDUCAO FUNDEB
1,917220102,Deducao de Cota-Parte do IPVA - FUNDEB,IPVA,DEDUCAO FUNDEB
2,917220104,Deducao de Cota-Parte do IPI sobre Exportacao ...,IPI ExportaCAo,DEDUCAO FUNDEB
3,917230000,Deducao de Transferencias dos Municipios,OUTRAS TRANSFERENCIAS,OUTRAS DEDUCOES
4,917213600,Deducao de Transferencia Financeira do ICMS De...,ICMS DesoneraCAo L.C. Nº 87.96,DEDUCAO FUNDEB


In [25]:
#Criar tabela auxiliar
cur.execute('''CREATE TABLE receitasDePara
               (numeroReceitaOrcamentariaUG_2 text,
               classificacaoSerec text,
               detalhamento text
               )
               ''')

In [26]:
#Inserir dados na tabela
receitas_tratadas_normalizadas = receitas_tratadas[['NUMERORECEITAORCAMENTARIAUG',	
                                                    'CLASSIFICACAO_SEREC', 
                                                    'DETALHAMENTO']]
receitas_tratadas_lista = receitas_tratadas_normalizadas.values.tolist()

try:
  cur.executemany("INSERT INTO receitasDePara (numeroReceitaOrcamentariaUG_2, classificacaoSerec, detalhamento) VALUES (?, ?, ?)", receitas_tratadas_lista)
  con.commit()
  print("Dados inseridos com sucesso.")
except Exception as e:
  print(str(e))
  print("Falha na inserção de dados.")
finally:
  pass

Dados inseridos com sucesso.


In [27]:
#Criar tabela principal
cur.execute('''CREATE TABLE transferencias
               (data text,
               ano text,
               origemReceita text, 
               deducaoReceita text,
               numeroReceitaOrcamentariaUG text,
               descricao text,
               valor real,
               FOREIGN KEY (numeroReceitaOrcamentariaUG) REFERENCES receitaDePara(numeroReceitaOrcamentariaUG_2))
               ''')

In [28]:
#Inserir dados na tabela
base_dados_lista = base_dados_gravar.values.tolist()

try:
  cur.executemany("INSERT INTO transferencias (data, ano, origemReceita, deducaoReceita, numeroReceitaOrcamentariaUG, descricao, valor) VALUES (?, ?, ?, ?, ?, ?, ?)", base_dados_lista)
  con.commit()
  print("Dados inseridos com sucesso.")
except Exception as e:
  print(str(e))
  print("Falha na inserção de dados.")
finally:
  con.close()

Dados inseridos com sucesso.


# ETAPA 02 - Organização dos dados para Previsão anual

## Salvar tabelas em variáveis

In [53]:
# Criar nova conexão e consultar tabelas gravadas no Banco de dados 
con = sqlite3.connect('baseTransferencias.db')
cur = con.cursor()

In [54]:
# Tabela Principal
cur.execute("SELECT * FROM transferencias")
tabela_principal = cur.fetchall()
tabela_principal = pd.DataFrame(tabela_principal)
tabela_principal.columns = base_dados_gravar.columns

In [55]:
# Tabela Auxiliar
cur.execute("SELECT * FROM receitasDePara")
tabela_auxiliar = cur.fetchall()
tabela_auxiliar = pd.DataFrame(tabela_auxiliar)
tabela_auxiliar.columns = receitas_tratadas_normalizadas.columns

In [56]:
tabela_principal

,COMPETENCIA,ANOREFERENCIA,ORIGEMRECEITA,DEDUCAORECEITA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,VALORLANCAMENTO
0,2012-01-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-4810177.11
1,2012-01-01,2012,Alienação de Bens,9,917220102,Dedução de Cota-Parte do IPVA - FUNDEB,-145288.61
2,2012-01-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-4222.80
3,2012-02-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-8771.62
4,2012-03-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-6409.60
...,...,...,...,...,...,...,...
14399,2022-02-01,2022,Transferências de Capital,0,24115031,Transfer?ncias de Recursos do Bloco de Manuten...,-3292391.42
14400,2022-04-01,2022,Transferências de Capital,0,24115091,Transfer?ncias de Recursos do Bloco de Manuten...,-1837875.59
14401,2022-05-01,2022,Transferências de Capital,0,24115091,Transfer?ncias de Recursos do Bloco de Manuten...,-980920.62
14402,2022-05-01,2022,Transferências de Capital,0,24115040,Transfer?ncias de Recursos do Bloco de Manuten...,-79.49


In [57]:
tabela_auxiliar

,NUMERORECEITAORCAMENTARIAUG,CLASSIFICACAO_SEREC,DETALHAMENTO
0,917220101,ICMS,DEDUCAO FUNDEB
1,917220102,IPVA,DEDUCAO FUNDEB
2,917220104,IPI ExportaCAo,DEDUCAO FUNDEB
3,917230000,OUTRAS TRANSFERENCIAS,OUTRAS DEDUCOES
4,917213600,ICMS DesoneraCAo L.C. Nº 87.96,DEDUCAO FUNDEB
...,...,...,...
287,17149901,None,None
288,17165011,None,None
289,13110203,None,None
290,17179901,None,None


In [58]:
con.close()

## Juntar tabelas e filtrar apenas Receita com Fundo de Participação dos Municípios

In [38]:
tabela_completa = None
tabela_completa = tabela_principal.merge(tabela_auxiliar, how='left', on='NUMERORECEITAORCAMENTARIAUG')

In [39]:
tabela_completa.shape

(15673, 9)

In [52]:
tabela_completa.tail()

,COMPETENCIA,ANOREFERENCIA,ORIGEMRECEITA,DEDUCAORECEITA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,VALORLANCAMENTO,CLASSIFICACAO_SEREC,DETALHAMENTO
15668,2022-02-01,2022,Transferências de Capital,0,24115031,Transfer?ncias de Recursos do Bloco de Manuten...,-3292391.42,NaN,NaN
15669,2022-04-01,2022,Transferências de Capital,0,24115091,Transfer?ncias de Recursos do Bloco de Manuten...,-1837875.59,NaN,NaN
15670,2022-05-01,2022,Transferências de Capital,0,24115091,Transfer?ncias de Recursos do Bloco de Manuten...,-980920.62,NaN,NaN
15671,2022-05-01,2022,Transferências de Capital,0,24115040,Transfer?ncias de Recursos do Bloco de Manuten...,-79.49,NaN,NaN
15672,2022-06-01,2022,Transferências de Capital,0,24115031,Transfer?ncias de Recursos do Bloco de Manuten...,-2462623.39,NaN,NaN


In [41]:
# Filtrar receita FPM
tabela_receita_FPM = tabela_completa.loc[tabela_completa['CLASSIFICACAO_SEREC'] == "FPM"]

In [42]:
tabela_receita_FPM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 180 to 14580
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COMPETENCIA                  306 non-null    object 
 1   ANOREFERENCIA                306 non-null    object 
 2   ORIGEMRECEITA                306 non-null    object 
 3   DEDUCAORECEITA               306 non-null    object 
 4   NUMERORECEITAORCAMENTARIAUG  306 non-null    object 
 5   DESCRICAO                    306 non-null    object 
 6   VALORLANCAMENTO              306 non-null    float64
 7   CLASSIFICACAO_SEREC          306 non-null    object 
 8   DETALHAMENTO                 306 non-null    object 
dtypes: float64(1), object(8)
memory usage: 23.9+ KB


In [43]:
# Transformar COMPETENCIA para Datetime
tabela_receita_FPM['COMPETENCIA'] = pd.to_datetime(tabela_receita_FPM['COMPETENCIA'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
tabela_receita_FPM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 180 to 14580
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   COMPETENCIA                  306 non-null    datetime64[ns]
 1   ANOREFERENCIA                306 non-null    object        
 2   ORIGEMRECEITA                306 non-null    object        
 3   DEDUCAORECEITA               306 non-null    object        
 4   NUMERORECEITAORCAMENTARIAUG  306 non-null    object        
 5   DESCRICAO                    306 non-null    object        
 6   VALORLANCAMENTO              306 non-null    float64       
 7   CLASSIFICACAO_SEREC          306 non-null    object        
 8   DETALHAMENTO                 306 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 23.9+ KB


In [45]:
tabela_receita_FPM.set_index('COMPETENCIA', inplace=True)
tabela_receita_FPM.head()

,ANOREFERENCIA,ORIGEMRECEITA,DEDUCAORECEITA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,VALORLANCAMENTO,CLASSIFICACAO_SEREC,DETALHAMENTO
COMPETENCIA,,,,,,,,
2012-03-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,4772635.69,FPM,PRINCIPAL
2012-03-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,4772635.69,FPM,PRINCIPAL
2012-07-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,4287083.00,FPM,PRINCIPAL
2012-07-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,4287083.00,FPM,PRINCIPAL
2012-01-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,5809328.85,FPM,PRINCIPAL


In [46]:
tabela_receita_FPM.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 306 entries, 2012-03-01 to 2021-06-01
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ANOREFERENCIA                306 non-null    object 
 1   ORIGEMRECEITA                306 non-null    object 
 2   DEDUCAORECEITA               306 non-null    object 
 3   NUMERORECEITAORCAMENTARIAUG  306 non-null    object 
 4   DESCRICAO                    306 non-null    object 
 5   VALORLANCAMENTO              306 non-null    float64
 6   CLASSIFICACAO_SEREC          306 non-null    object 
 7   DETALHAMENTO                 306 non-null    object 
dtypes: float64(1), object(7)
memory usage: 21.5+ KB


In [59]:
# Resumir dados por Competência (soma Rubricas Orçamentárias FPM Bruto - Dedução FUNDEB FPM)
tabela_receita_FPM = tabela_receita_FPM.resample("M").sum()

In [61]:
tabela_receita_FPM.index

DatetimeIndex(['2012-01-31', '2012-02-29', '2012-03-31', '2012-04-30',
               '2012-05-31', '2012-06-30', '2012-07-31', '2012-08-31',
               '2012-09-30', '2012-10-31',
               ...
               '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30',
               '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31',
               '2021-11-30', '2021-12-31'],
              dtype='datetime64[ns]', name='COMPETENCIA', length=120, freq='M')

In [62]:
tabela_receita_FPM.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 120 entries, 2012-01-31 to 2021-12-31
Freq: M
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   VALORLANCAMENTO  120 non-null    float64
dtypes: float64(1)
memory usage: 1.9 KB


In [64]:
# Gerar tabela para validação dos dados em excel
tabela_receita_FPM.to_csv('/content/tabela_receita_FPM.csv')

# ETAPA 03 - Divisão dos dados (Treino/teste)

In [71]:
train = tabela_receita_FPM.iloc[0:-12, 0]
test = tabela_receita_FPM.iloc[-12:, 0]

In [72]:
train

COMPETENCIA
2012-01-31     5809328.85
2012-02-29     7027265.76
2012-03-31     4772635.69
2012-04-30     6015086.32
2012-05-31     6727448.46
                 ...     
2020-08-31     4934918.72
2020-09-30     3975120.48
2020-10-31     5348907.62
2020-11-30     7089571.88
2020-12-31    11356239.86
Freq: M, Name: VALORLANCAMENTO, Length: 108, dtype: float64

In [73]:
test

COMPETENCIA
2021-01-31     7995406.81
2021-02-28    10479951.06
2021-03-31     7024491.01
2021-04-30     7343573.11
2021-05-31     8825963.67
2021-06-30     7633088.13
2021-07-31    11373069.84
2021-08-31     8338885.51
2021-09-30     6548838.74
2021-10-31     7297591.74
2021-11-30     9477576.86
2021-12-31    15177161.97
Freq: M, Name: VALORLANCAMENTO, dtype: float64

# ETAPA 04 - Aplicar Modelos de previsão

## Regressão linear

In [ ]:
lr = LinearRegression()

In [ ]:
model = lr.fit(X = tabela_receita_FPM_anual['ANO'], y = tabela_receita_FPM_anual['VALORLANCAMENTO'])

KeyError: ignored

In [ ]:

# Confirmar valor com arrecadação Município
# Consolidar por ano
# Aplicar modelos > função?